# Big Data in Finance: Part III <BR><BR>Homework 2: Momentum Factor
    
**Instructor** <BR>
Ritt Keerati, ritt.keerati@gsb.columbia.edu
    
**TA's:** <BR>
Meha Sadasivam, MSadasivam21@gsb.columbia.edu <BR>
Daheng Yang, dyang22@gsb.columbia.edu

**Due:** Thursday, April 1 at 11:59pm. Late submission will *not* be accepted. 

**Goal:** Last class we went through the building blocks to create investable portfolios from characteristics that forecast future returns. In this homework you are going to explore a different characteristic that has traditionally been associated with future returns: the past performance. The momemtum strategy is the one that invests in a portfolio that goes long stock with high past performance (high past returns) and short the ones with low performance (low past returns). Your goal in this homework is to calculate the cumulative returns of the momentum portolio from 1970.

**Since we will cover the momentum factor in more details in our last lecture, for this assignment you will focus your efforts on portfolio sort (Part 4) and comparing your results to those from Fama and French (Part 5).**

**Delivery:** Upload on canvas a .zip file with your code (it might include .ipynb or .py) and your answers (.html notebook). 

## Setup
For this homework you need the fire_pytools package that you can find in the Bitbucket repository [here](https://bitbucket.org/liramota/fire_pytools/). 

fire_pytools stands for (FI)nance (RE)search python tools and contains a bunch of useful functions, including data download, portfolio sorts, etc.

You should save the fire_pytools in folder in your computer and adapt the code below with the path for the package.

If you decide to download the fire_pytools to separate folder and you are using PyCharm, the best way to use an external library in pycharm is by adding the library PATH to your project. Check [here](https://www.jetbrains.com/help/pycharm/configuring-content-roots.html) if you have problems.

In [2]:
%matplotlib inline 

# Packages 
import sys
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

import time
from time import strptime, strftime
from pandas.tseries.offsets import MonthEnd

# Setups
pd.set_option("display.max_rows", 100) # max number or rows to be displayed 
plt.rcParams['figure.figsize'] = [10, 6]
idx = pd.IndexSlice

In [3]:
# Packages from fire_pytools
sys.path.append("/Users/rk2941/Dropbox/courses/big_data_2022/fire_pytools")

# Data Import
from data_setup import stock_monthly
from data_setup import stock_annual

# Functions 
from utils.monthly_date import *
from portools.find_breakpoints import find_breakpoints
from portools.sort_portfolios import sort_portfolios
from import_kf.kf_factors import *
from utils.post_event_nan import *

## Download Data

In [4]:
# This is the same data we used in class. It use data download from CRSP
#mdata = stock_monthly.main()
#mdata.to_pickle(path = '/Users/rk2941/Dropbox/courses/big_data_2022/data/stock_monthly.pkl')
mdata = pd.read_pickle('/Users/rk2941/Dropbox/courses/big_data_2022/data/stock_monthly.pkl')

## Calculate past 11-1 returns

In order to calculate the momentum factor, first we need to calculate the "past cumulative returns". This is the only characteristic that you need to construct yourself. But the code is already here for you.

The original Carhart (1997) paper offers many windows of cumulating returns and months to be skipped before the formation date. In the code we have adopted the notation ret_{window}_{skip}, where {window} is the number of months to cumulate the returns and {skip} is the number of months to be skipped before this return is used for sorting.

𝑟𝑒𝑡\_11\_1  is the 11 months cumulative returns, skipping 1 month, or, it is cumulative returns from 12 months before to one month before the formation date. This is the characteristic we are going to use to calculate the MOM factor.

Be careful, the ret_{window}_{skip} notation is not to be confused with the Ken French notation. Since we are going to hold it for the following month, when Ken French report returns, he calls them: "portfolios formed on prior (2-12) return". Again, this for portfolio holding returns, not sorts.

Make sure to be mindful about missing returns. We are going to follow the convention in Daniel and Moskowitz, JFE 2016, that says: "We require that a firm ... [has] a minimum of eight monthly returns over the past 11 months, skipping the most recent month". That means that if there are less than eight valid returns in the 11-period, we should set ret_11_1 equal to missing.

In [5]:
# Clean Data
## Data start in 2010
mdata = mdata[mdata.date>='2010-01-01']
## Share type - select only common stocks 
mdata = mdata[mdata.shrcd.isin([10, 11])]
## Select stocks traded on the major stock exchanges
mdata = mdata[mdata.exchcd.isin([1, 2, 3])]
## Set index
mdata.set_index(['permno','date'], inplace=True)

In [6]:
# Calculate excess returns
mdata['exret'] = mdata['retadj'] - mdata['rf']

In [9]:
# Calculate past returns 
def calculate_cumulative_returns(mdata, tt, min_periods):
    """
    Calculate past returns for momentum stratagy

    Parameters:
    ------------
    mdata: data frame
        crsp monthly data with cols permno, date as index.
    tt: int
        number of periods to cumulate retuns
    min_periods: int
    """
    start_time = time.time()
    required_cols = ['retadj']

    assert set(required_cols).issubset(mdata.columns), "Required columns: {}.".format(', '.join(required_cols))

    df = mdata[required_cols].copy()

    # Before applying the function calculate_cumulative_returns you need to make sure 
    # we don't have missing dates. Missing dates would lead to the wrong number of periods 
    # to cumulate returns.

    # Resample data 
    # CRSP data has skipping months.
    # Create line to missing  months to facilitate the calculation of lag/past returns
    df.reset_index(inplace=True)
    df['edate'] = df['date'] + MonthEnd(0)
    df.sort_values(['permno', 'edate'], inplace=True)
    pk_integrity(df, ['permno', 'edate'])
    df.set_index(['edate'], inplace=True)
    # Resample to take care of missing months
    sdf = df[['permno', 'retadj']].groupby('permno').resample('M').mean().drop(columns='permno')
    sdf.reset_index(inplace=True)
    print("Time to resample data: %s seconds" % str(time.time() - start_time))
    print("We have added %s rows to the data" % str(sdf.shape[0] - df.shape[0]))

    # Cumulative Return (adjusted) in tt months
    sdf['gret'] = sdf['retadj'].fillna(0)+1
    sdf['ret_notnull'] = sdf['retadj'].notnull()

    cret = sdf.groupby(['permno']).gret.rolling(window=tt, min_periods=1).apply(np.prod, raw=True)
    cret.reset_index(drop=True, inplace=True)

    nona_count = sdf.groupby(['permno']).ret_notnull.rolling(window=tt, min_periods=1).apply(np.sum, raw=True)
    nona_count.reset_index(drop=True, inplace=True)

    cret[nona_count<min_periods] = np.nan
    sdf['ret' + str(tt)] = cret

    # Delete rows that were not in the original data set
    df.reset_index(inplace=True)
    fdata = pd.merge(df,
             sdf[['edate', 'permno','ret' + str(tt)]],
             on = ['edate', 'permno'])         
    fdata.set_index(['permno','date'],inplace = True)
    x = fdata['ret' + str(tt)]

    print("Time to calculate %d months past returns: %s seconds" % (tt, str(round(time.time() - start_time, 2))))

    return x

In [8]:
## Calculate 11 months returns
mdata['ret11'] = calculate_cumulative_returns(mdata, 11, 8)

Time to resample data: 39.52799415588379 seconds
We have added 1909 rows to the data
Time to calculate 11 months past returns: 49.07 seconds


In [ ]:
np.mean(mdata.ret11.notnull())

In [ ]:
## Skip one month 
mdata['ret11_1'] = mdata.groupby(['permno']).ret11.shift(1)

In [ ]:
# Create MEsum: when sorting by size this is the ME to be used
mdata['mesum'] = mdata.groupby(['date', 'permco']).me.transform(np.sum, min_count=1)

## Portfolios Sorts

Your task in this question is to do portfolio sorting. Use the function "sort_portfolios" to sort stocks into portfolio buckets.

Check [Ken French's website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html) for Mom factor description. 

Every end of month, sort stocks into 2 X 3 buckets according to ME and ret_11_1. We follow Ken French website, and use 50th percentile for ME and 30th and 70th percentiles  for ret_11_1.

Calculate the ME value weighted returns for each of the 6 portfolios sorted. Make sure that, for every month, you use information that is available for investors at the time of portfolio formation. Plot the cumulative returns for each of them.

Calculate the MOM returns. From Ken French website: Mom is the average return on the two high prior return portfolios minus the average return on the two low prior return portfolios,

$$Mom =	\frac{1}{2}(Small High + Big High) - \frac{1}{2}(Small Low + Big Low).$$	 

Plot the cumulative returns and report the average returns, standard devation and Sharpe-Ratio. Is your MOM portfolio returns similar to the one published in the Ken French website? Report the correlation.

## Comparison to Fama and French
Download Ken French's **Momentum Factor (Mom)** from [Ken French's website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html). Compare your momentum factor to the data from Fama and French. Report the correlation between the two series, and plot the two series on the same chart for comparison.